
# Lineare Regression mit einem Merkmal

Dieses Notebook zeigt Schritt für Schritt, wie wir mit scikit-learn eine einfache lineare Regression aufbauen. Der Fokus liegt darauf, die wichtigsten Konzepte zu verstehen und nicht auf möglichst ausgefeilten Tricks.



Wir gehen in vier kurzen Schritten vor:

1. Daten verstehen und visualisieren.
2. Ein Regressionsmodell trainieren.
3. Vorhersagen erzeugen und Kennzahlen berechnen.
4. Ergebnisse visualisieren und interpretieren.


In [ ]:

import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



## 1. Daten kennenlernen

Wir verwenden `data/data-ads.csv`. Jede Zeile beschreibt, wie viel Budget (`ads`) für Werbung ausgegeben wurde und wie hoch die Verkäufe (`sales`) waren. So können wir untersuchen, ob sich höhere Ausgaben lohnen.


In [ ]:

df = pd.read_csv("data/data-ads.csv")
df.head()



Ein erster Blick auf Basisstatistiken hilft zu verstehen, in welchen Größenordnungen sich die Variablen bewegen und ob Ausreißer auffallen.


In [ ]:

df.describe().T



Die Korrelation ist ein grober Indikator dafür, wie stark beide Variablen zusammenhängen.


In [ ]:

df.corr(numeric_only=True).round(2)



Ein Streudiagramm macht sichtbar, ob sich ein ungefähr linearer Trend erkennen lässt.


In [ ]:

scatter = (
    alt.Chart(df)
    .mark_circle(size=80, color="#1f77b4")
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("sales", title="Verkäufe"))
    .properties(title="Werbung vs. Verkäufe")
)
trend = scatter.transform_regression("ads", "sales").mark_line(color="#d62728", size=3)
scatter + trend



## 2. Modell aufbauen

Wir nutzen nur ein Merkmal (`ads`). Das Ziel ist `sales`. Die lineare Regression schätzt eine Gerade, die den Trend möglichst gut beschreibt.


In [ ]:

feature = "ads"
target = "sales"

X = df[[feature]]
y = df[target]



Jetzt trainieren wir das Modell. `fit` passt Achsenabschnitt und Steigung so an, dass der Fehler minimiert wird.


In [ ]:

model = LinearRegression()
model.fit(X, y)



Die Parameter des Modells lassen sich einfach interpretieren: die Steigung zeigt, wie stark `sales` pro zusätzlicher Einheit in `ads` steigt.


In [ ]:

coef_table = pd.DataFrame(
    {
        "Parameter": ["Achsenabschnitt", "Steigung"],
        "Wert": [model.intercept_, model.coef_[0]],
    }
)
coef_table



## 3. Vorhersagen und Kennzahlen

Wir ergänzen den Datensatz um die Vorhersage und vergleichen sie mit den echten Verkäufen.


In [ ]:

df = df.assign(prediction=model.predict(X))
df[["ads", "sales", "prediction"]].head()



Zur Bewertung nutzen wir die mittlere quadratische Abweichung (MSE), ihre Quadratwurzel (RMSE) und das Bestimmtheitsmaß $R^2$.


In [ ]:

mse = mean_squared_error(y, df["prediction"])
rmse = mse ** 0.5
r2 = r2_score(y, df["prediction"])

metrics = pd.Series({"MSE": mse, "RMSE": rmse, "R^2": r2}, name="Modellgüte")
metrics.to_frame("Wert").round(3)



Eine gemeinsame Grafik zeigt, wie dicht die Modellgerade an den Beobachtungen liegt.


In [ ]:

df_sorted = df.sort_values(feature)
line = (
    alt.Chart(df_sorted)
    .mark_line(color="#d62728", size=3)
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("prediction", title="Vorhersage"))
)
scatter + line



## 4. Merksatz

- Eine einfache lineare Regression reicht, wenn nur ein klarer Einflussfaktor existiert.
- Visualisierungen helfen, Annahmen (z. B. Linearität) zu prüfen.
- Metriken wie RMSE oder $R^2$ liefern eine Zahl, mit der Modelle verglichen werden können.
